In [ ]:
import pandas as pd
from pprint import pprint
from comprehend_detect import ComprehendDetect
from detect_pi_si import *
from PIDetectAnalysis import *
import boto3
from botocore.exceptions import ClientError
import warnings
warnings.filterwarnings('ignore')
comprehend_client = boto3.client("comprehend")

### Run Comprehend detection and redaction on 1000 rows of sample data in S3

In [ ]:
df = pd.read_csv("s3://pearsoncomprehend/comprehendData/pii_test_faker.csv",delimiter=",")
df.dropna()

#instantiate wrapper class
comp_detect = ComprehendDetect(comprehend_client)

#return PII detections
df_pi = detect_pi(df,comp_detect)

#redact PII detections
df_redacted = redact_df(df,df_pi)

#rescan for excluded names
df_names = detect_names(df_redacted,comp_detect)

#redact names
df_redacted = redact_df(df_redacted,df_names)


generator = PIDetectionReportGenerator("s3://pearsoncomprehend/comprehendData/detectedPI.csv")
generator.generate_markdown_report()

Markdown report generated successfully: reports/PI_detection_report.md


'reports/PI_detection_report.md'

### Write result tables to S3

In [ ]:
df_pi.to_csv("s3://pearsoncomprehend/comprehendData/detectedPI.csv",index=False)
df_names.to_csv("s3://pearsoncomprehend/comprehendData/detectedNames.csv",index=False)
df_redacted.to_csv("s3://pearsoncomprehend/comprehendData/fakerRedacted.csv",index=False)